## Asking the Right Question
Use the Machine Learning Workflow to process and transform DOT data to create a prediction model. This model must predict whether a flight would arrive 15+ minutes after the scheduled arrival time with 70+% accuracy.

## Preparing Data

### Getting Data
Download 2015 January raw data in csv file from the following URL:
http://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236&DB_Short_Name=On-Time

The downloaded data stored in [raw_201501.csv](data/raw_201501.csv).

### Loading Data

In [1]:
origData <- read.csv2('.\\data\\raw_201501.csv', sep=",", header=TRUE, stringsAsFactors = FALSE)
nrow(origData)

[1] 469968

use the subset of data from major airports only

In [2]:
airports <- c('ATL', 'LAX', 'ORD', 'DFW', 'JFK', 'SFO', 'CLT', 'LAS', 'PHX')
filteredData <- subset(origData, DEST %in% airports & ORIGIN %in% airports)
nrow(filteredData)

[1] 32716

### Cleaning Data

In [3]:
head(filteredData, 2)

DAY_OF_MONTH,DAY_OF_WEEK,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,TAIL_NUM,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,...,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE,X
1,4,AA,19805,AA,N787AA,1,12478,1247802,JFK,...,LAX,855,0.00,0900-0959,1237,0.00,0.00,0.00,2475.00,NA
1,4,AA,19805,AA,N795AA,2,12892,1289203,LAX,...,JFK,856,0.00,0900-0959,1651,0.00,0.00,0.00,2475.00,NA


In [4]:
tail(filteredData, 2)

,DAY_OF_MONTH,DAY_OF_WEEK,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,TAIL_NUM,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,...,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE,X
469666,31,6,WN,19393,WN,N659SW,3841,14771,1477101,SFO,...,PHX,1109,0.00,1100-1159,1417,0.00,0.00,0.00,651.00,NA
469667,31,6,WN,19393,WN,N218WN,4481,14771,1477101,SFO,...,PHX,1426,0.00,1400-1459,1721,0.00,0.00,0.00,651.00,NA


#### Remove unuse column

X column always had value "NA", remove this column by set it to NULL

In [5]:
filteredData$X <- NULL
tail(filteredData, 2)
head(filteredData, 2)

,DAY_OF_MONTH,DAY_OF_WEEK,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,TAIL_NUM,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,...,DEST_AIRPORT_SEQ_ID,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE
469666,31,6,WN,19393,WN,N659SW,3841,14771,1477101,SFO,...,1410702,PHX,1109,0.00,1100-1159,1417,0.00,0.00,0.00,651.00
469667,31,6,WN,19393,WN,N218WN,4481,14771,1477101,SFO,...,1410702,PHX,1426,0.00,1400-1459,1721,0.00,0.00,0.00,651.00


DAY_OF_MONTH,DAY_OF_WEEK,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,TAIL_NUM,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,...,DEST_AIRPORT_SEQ_ID,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE
1,4,AA,19805,AA,N787AA,1,12478,1247802,JFK,...,1289203,LAX,855,0.00,0900-0959,1237,0.00,0.00,0.00,2475.00
1,4,AA,19805,AA,N795AA,2,12892,1289203,LAX,...,1247802,JFK,856,0.00,0900-0959,1651,0.00,0.00,0.00,2475.00


#### Check correlated values

In [6]:
cor(filteredData[c('ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID')])
cor(filteredData[c('DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID')])

,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID
ORIGIN_AIRPORT_ID,1,1
ORIGIN_AIRPORT_SEQ_ID,1,1


,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID
DEST_AIRPORT_ID,1,1
DEST_AIRPORT_SEQ_ID,1,1


Drop correlated columns that doesn't provide new information

In [7]:
filteredData$ORIGIN_AIRPORT_SEQ_ID <- NULL
filteredData$DEST_AIRPORT_SEQ_ID <- NULL
head(filteredData, 2)

DAY_OF_MONTH,DAY_OF_WEEK,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,TAIL_NUM,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,DEST_AIRPORT_ID,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE
1,4,AA,19805,AA,N787AA,1,12478,JFK,12892,LAX,855,0.00,0900-0959,1237,0.00,0.00,0.00,2475.00
1,4,AA,19805,AA,N795AA,2,12892,LAX,12478,JFK,856,0.00,0900-0959,1651,0.00,0.00,0.00,2475.00


#### Check duplicate columns that have identical values

In [8]:
mismatched <- filteredData[filteredData$CARRIER != filteredData$UNIQUE_CARRIER,]
nrow(mismatched)

[1] 0

It is identical, remove the column

In [9]:
filteredData$UNIQUE_CARRIER <- NULL
head(filteredData, 2)

DAY_OF_MONTH,DAY_OF_WEEK,AIRLINE_ID,CARRIER,TAIL_NUM,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,DEST_AIRPORT_ID,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE
1,4,19805,AA,N787AA,1,12478,JFK,12892,LAX,855,0.00,0900-0959,1237,0.00,0.00,0.00,2475.00
1,4,19805,AA,N795AA,2,12892,LAX,12478,JFK,856,0.00,0900-0959,1651,0.00,0.00,0.00,2475.00


### Molding Data

#### Remove unlabeled rows

Remove rows that is no value or with value "NA" or "" for target/label variable such as Arr_Del15 and Dep_Del15

In [10]:
onTimeData <- filteredData[!is.na(filteredData$ARR_DEL15) & filteredData$ARR_DEL15!="" & !is.na(filteredData$DEP_DEL15) & filteredData$DEP_DEL15!="",]
nrow(filteredData)
nrow(onTimeData)

[1] 32716

[1] 32124

#### Change data types

In [11]:
onTimeData$DISTANCE <- as.integer(onTimeData$DISTANCE)
onTimeData$CANCELLED <- as.integer(onTimeData$CANCELLED)
onTimeData$DIVERTED <- as.integer(onTimeData$DIVERTED)
onTimeData$ARR_DEL15 <- as.factor(onTimeData$ARR_DEL15)
onTimeData$DEP_DEL15 <- as.factor(onTimeData$DEP_DEL15)
onTimeData$ORIGIN_AIRPORT_ID <- as.factor(onTimeData$ORIGIN_AIRPORT_ID)
onTimeData$DEST_AIRPORT_ID <- as.factor(onTimeData$DEST_AIRPORT_ID)
onTimeData$DAY_OF_WEEK <- as.factor(onTimeData$DAY_OF_WEEK)
onTimeData$ORIGIN <- as.factor(onTimeData$ORIGIN)
onTimeData$DEST <- as.factor(onTimeData$DEST)
onTimeData$DEP_TIME_BLK <- as.factor(onTimeData$DEP_TIME_BLK)
onTimeData$CARRIER <- as.factor(onTimeData$CARRIER)
head(onTimeData, 10)

,DAY_OF_MONTH,DAY_OF_WEEK,AIRLINE_ID,CARRIER,TAIL_NUM,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,DEST_AIRPORT_ID,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE
1,1,4,19805,AA,N787AA,1,12478,JFK,12892,LAX,855,0.00,0900-0959,1237,0.00,0,0,2475
2,1,4,19805,AA,N795AA,2,12892,LAX,12478,JFK,856,0.00,0900-0959,1651,0.00,0,0,2475
3,1,4,19805,AA,N798AA,3,12478,JFK,12892,LAX,1226,0.00,1200-1259,1548,0.00,0,0,2475
4,1,4,19805,AA,N799AA,4,12892,LAX,12478,JFK,1214,0.00,1200-1259,2033,0.00,0,0,2475
9,1,4,19805,AA,N792AA,9,12478,JFK,12892,LAX,649,0.00,0700-0759,1026,0.00,0,0,2475
10,1,4,19805,AA,N796AA,10,12892,LAX,12478,JFK,2150,0.00,2100-2159,544,0.00,0,0,2475
11,1,4,19805,AA,N792AA,12,12892,LAX,12478,JFK,1114,0.00,1100-1159,1935,0.00,0,0,2475
13,1,4,19805,AA,N783AA,15,12478,JFK,14771,SFO,755,0.00,0800-0859,1228,1.00,0,0,2586
14,1,4,19805,AA,N783AA,16,14771,SFO,12478,JFK,1316,1.00,1200-1259,2149,1.00,0,0,2586
16,1,4,19805,AA,N786AA,19,12478,JFK,12892,LAX,1055,0.00,1100-1159,1413,0.00,0,0,2475


In [12]:
tapply(onTimeData$ARR_DEL15, onTimeData$ARR_DEL15, length)
6460/(6460 + 25664)

0.00  1.00 
25664  6460

[1] 0.2010958

20% delay flights based on the above calculation.

## Selecting algorithm
The selected initial algorithm is Logistic Regression as it is simple(easy to understand), fast(up to 100x faster) and stable to data changes.

Then, switch to Random Forest to improve prediction result.

## Training the model

 * Training the model using caret package

In [13]:
# Download and install caret locally
# install.packages("caret", dependencies=TRUE, repos='http://cran.us.r-project.org')

# Load caret
library(caret)

Warning message:
"package 'caret' was built under R version 3.3.1"Loading required package: lattice
Loading required package: ggplot2


 * Set the seed so that the random number generated in the same sequence to yield the same repeatable/reproducable training results

In [14]:
set.seed(122515)

* Retrieve feature columns only

In [15]:
# set the columns we are going to use to train algorithm
featureCols <- c("ARR_DEL15", "DAY_OF_WEEK", "CARRIER", "DEST","ORIGIN","DEP_TIME_BLK")

# created filtered version of onTimeData dataframe
onTimeDataFiltered <- onTimeData[,featureCols]

 * Retrieve 70% of data for training

In [16]:
# create vector contain row indicies to put into the training data frames
inTrainRows <- createDataPartition(onTimeDataFiltered$ARR_DEL15, p=0.70, list=FALSE)
# check the row IDs
head(inTrainRows,10)
# Create the training data frame
trainDataFiltered <- onTimeDataFiltered[inTrainRows,]

Resample1
2
4
5
7
10
11
12
14
15
16


* Retrieve the remaining 30% of data for testing

In [17]:
# Create the testing data frame.  Notice the prefix "-" 
testDataFiltered <- onTimeDataFiltered[-inTrainRows,]

* Check split data

In [18]:
#   Should be 70%
nrow(trainDataFiltered)/(nrow(testDataFiltered) + nrow(trainDataFiltered))
#   Should be 30%
nrow(testDataFiltered)/(nrow(testDataFiltered) + nrow(trainDataFiltered))

[1] 0.7000062

[1] 0.2999938

 * Train the model with train data

In [19]:
#  Logistic Regression
logisticRegModel <- train(ARR_DEL15 ~ ., data=trainDataFiltered, method="glm", family="binomial", trControl=trainControl(method="cv", number=10, repeats=10))

## Testing/Evaluating the model

 * Predict using trained model against test data

In [20]:
logRegPrediction <- predict(logisticRegModel, testDataFiltered)

 * Get detailed statistics of prediction versus actual via Confusion Matrix 

In [21]:
logRegConfMat <- confusionMatrix(logRegPrediction, testDataFiltered[,"ARR_DEL15"])
logRegConfMat

Confusion Matrix and Statistics

          Reference
Prediction 0.00 1.00
      0.00 7676 1907
      1.00   23   31
                                          
               Accuracy : 0.7997          
                 95% CI : (0.7916, 0.8077)
    No Information Rate : 0.7989          
    P-Value [Acc > NIR] : 0.4254          
                                          
                  Kappa : 0.0204          
 Mcnemar's Test P-Value : <2e-16          
                                          
            Sensitivity : 0.9970          
            Specificity : 0.0160          
         Pos Pred Value : 0.8010          
         Neg Pred Value : 0.5741          
             Prevalence : 0.7989          
         Detection Rate : 0.7965          
   Detection Prevalence : 0.9944          
      Balanced Accuracy : 0.5065          
                                          
       'Positive' Class : 0.00            
                                          

### Improving performance with Random Forest
We use the Random Forest algorithm which creates multiple decision trees and uses bagging to improve performance

In [23]:
#  install the package - this only needs to be done once.  After the package is installed
#  comment out this line unless you really want the latest version of the package to be downloaded
#  and installed
install.packages('randomForest', dependencies=TRUE, repos='http://cran.us.r-project.org')

#  load the random forest library into the current session
library(randomForest)

package 'randomForest' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\limcheek\AppData\Local\Temp\RtmpeaI5cL\downloaded_packages


Warning message:
"package 'randomForest' was built under R version 3.3.1"randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.

Attaching package: 'randomForest'

The following object is masked from 'package:ggplot2':

    margin



 * Create random forest predictive model

In [ ]:
# This code will run for a while!  It ran for 8 minutes on a system with a i7-4790K, 16 GB of memory, and a 500 GB SSD.
rfModel <- randomForest(trainDataFiltered[-1], trainDataFiltered$ARR_DEL15, proximity = TRUE, importance = TRUE)
rfModel

 * Make prediction using the random forest model and check metrics

In [ ]:
#   Random Forest
rfValidation <- predict(rfModel, testDataFiltered)
#    Get detailed statistics of prediction versus actual via Confusion Matrix 
rfConfMat <- confusionMatrix(rfValidation, testDataFiltered[,"ARR_DEL15"])
rfConfMat